# GEOEventFusion — Developer Sandbox

Experimental scratch notebook for iterating on individual pipeline components  
without running the full pipeline. Use this to:

- Test individual agent logic in isolation
- Inspect raw GDELT API responses
- Prototype new analysis functions
- Debug specific pipeline phases

**This notebook is NOT the canonical entry point.** See `quickstart.ipynb` for production use.

## Setup

In [1]:
!git clone https://github.com/dshipley71/GEOEventFusion.git

Cloning into 'GEOEventFusion'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 203 (delta 36), reused 140 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (203/203), 253.75 KiB | 3.13 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [2]:
%cd GEOEventFusion/
%ls

/content/GEOEventFusion
AGENTS.md     DIRECTORY_STRUCTURE.md  outputs/              scripts/
CHANGELOG.md  docs/                   pyproject.toml        skills.md
CLAUDE.md     geoeventfusion/         README.md             tests/
config/       LICENSE                 requirements-dev.txt
data/         notebooks/              requirements.txt


In [4]:
!pip install -r requirements-dev.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.2/455.2 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 146.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
%pwd

'/content/GEOEventFusion'

In [5]:
import sys
import json
import logging
from pathlib import Path

# Ensure project root is on path when running from notebooks/
_ROOT = Path().resolve().parent
if str(_ROOT) not in sys.path:
    sys.path.insert(0, str(_ROOT))

logging.basicConfig(level=logging.INFO, format='%(levelname)-8s %(name)s — %(message)s')
print(f'Project root: {_ROOT}')

Project root: /content


In [7]:
from dotenv import load_dotenv
load_dotenv()

from config.settings import PipelineConfig

# Minimal test config — no real API calls
config = PipelineConfig(
    query='Houthi Red Sea attacks',
    days_back=30,
    llm_backend='ollama',
    max_records=50,
    test_mode=True,
    log_level='DEBUG',
)
print('Config loaded:', config.query)

Config loaded: Houthi Red Sea attacks


In [9]:
import rich

# TODO Fix default ollama host to ue cloud instead of local

config.ollama_host = "http://ollama.com"
rich.print(config)

PipelineConfig(
    query='Houthi Red Sea attacks',
    days_back=30,
    max_records=50,
    llm_backend='ollama',
    anthropic_model='claude-sonnet-4-6',
    ollama_model='gemma3:27b',
    ollama_host='http://ollama.com',
    llm_temperature=0.1,
    llm_max_tokens=2048,
    llm_min_max_tokens=256,
    anthropic_api_key=None,
    acled_api_key=None,
    acled_email=None,
    spike_z_threshold=1.5,
    max_spikes=10,
    spike_backfill_hours=48,
    domain_cap_pct=0.2,
    timeline_smooth=3,
    repeat_threshold=3,
    near_window=15,
    near_min_term_length=5,
    tone_negative_threshold=-5.0,
    toneabs_threshold=8.0,
    gdelt_stagger_seconds=0.75,
    gdelt_max_workers=2,
    gdelt_max_retries=5,
    gdelt_backoff_base=2.0,
    gdelt_request_timeout=30,
    source_country_filter=None,
    source_lang_filter=None,
    authoritative_domains=[],
    visual_imagetags=[],
    enable_visual_intel=False,
    enable_word_clouds=False,
    rss_feed_list=[],
    rss_max_articles_per_spike=50,
    rss_time_window_hours=48,
    rss_request_timeout=15,
    rss_dedup_threshold=0.85,
    ground_truth_sources=[],
    ground_truth_country_filter=[],
    ground_truth_event_types=[],
    custom_dataset_path=None,
    custom_dataset_format='csv',
    fusion_weights=FusionWeights(temporal=0.25, geographic=0.25, actor=0.2, semantic=0.2, event_type=0.1),
    fusion_temporal_window_hours=72,
    fusion_geographic_threshold_km=200.0,
    max_confidence=0.82,
    min_citations=3,
    min_panel_confidence=0.25,
    validation_title_similarity_threshold=0.55,
    validation_ground_truth_similarity_threshold=0.65,
    validation_custom_match_threshold=0.5,
    validation_date_delta_days=7,
    validation_url_timeout=10,
    validation_min_corroboration=2,
    actor_hub_top_n=5,
    actor_broker_ratio_threshold=1.5,
    actor_pagerank_max_iter=200,
    visual_staleness_hours=72,
    output_root='outputs/runs',
    log_level='DEBUG',
    test_mode=True
)

## Test GDELT Client

In [33]:
# Direct GDELT API call — inspect raw response
from geoeventfusion.clients.gdelt_client import GDELTClient

client = GDELTClient(
    max_retries=config.gdelt_max_retries,
    backoff_base=config.gdelt_backoff_base,
    request_timeout=config.gdelt_request_timeout,
)

# TODO: Fix fetch to allow for timespan option vs start/end date

# Fetch a small article list
response = client.fetch(
    query='ICE protests in Minneapolis',
    mode='ArtList',
    max_records=100,
    sort='DateDesc',
    start_date="20260101000000", # YYYYMMDDHHMMSS or YYYY-MM-DD format
    end_date="20260227000000", # YYYYMMDDHHMMSS or YYYY-MM-DD format,
    # timespan='7d',
)

print('GDELTClient fetch call to run a live query COMPLETED)')

GDELTClient fetch call to run a live query COMPLETED)


In [34]:
print(f"Number of Articles: {len(response["articles"])}\n")

print(json.dumps(response, indent=2))

TypeError: 'NoneType' object is not subscriptable

## Test Spike Detector

In [29]:
from geoeventfusion.analysis.spike_detector import detect_spikes
from geoeventfusion.models.events import TimelineStep
from collections import Counter

articles = response.get("articles", [])

# ── Parse seendate → YYYY-MM-DD ──────────────────────────────────────────────
def _parse_date(raw: str) -> str:
    """Normalise GDELT seendate variants to YYYY-MM-DD.
    Handles: '20260220T120000Z', '2026-02-20', '20260220120000'.
    """
    s = raw.replace("-", "").replace("T", "").replace("Z", "").strip()
    # s is now digits only, at least 8
    return f"{s[:4]}-{s[4:6]}-{s[6:8]}"

# ── Aggregate article count per calendar day ──────────────────────────────────
date_counts: Counter = Counter()
for art in articles:
    raw_date = art.get("seendate", "")
    if raw_date:
        try:
            date_counts[_parse_date(raw_date)] += 1
        except (ValueError, IndexError):
            pass  # skip malformed dates

if not date_counts:
    print("No date data found in response — check that Cell 10 ran successfully.")
else:
    # Build sorted TimelineStep list (one entry per day)
    steps = [
        TimelineStep(date=d, value=float(c))
        for d, c in sorted(date_counts.items())
    ]

    print(f"Timeline built from {len(articles)} articles across {len(steps)} day(s):")
    for s in steps:
        print(f"  {s.date}  articles={int(s.value)}")

    if len(steps) >= 3:
        spikes = detect_spikes(steps, z_threshold=1.5)
        print(f"\nDetected {len(spikes)} spike(s):")
        for s in spikes:
            print(f"  [{s.rank}] {s.date}  Z={s.z_score:.2f}  vol={s.volume}")
    else:
        print("\n(Need ≥3 data points for spike detection — fetch more articles or widen the date range.)")

# from geoeventfusion.analysis.spike_detector import detect_spikes
# from geoeventfusion.models.events import TimelineStep

# # Build a synthetic timeline with one clear spike
# steps = [
#     TimelineStep(date=f'2024-01-{i:02d}', value=2.0)
#     for i in range(1, 28)
# ]
# steps[14] = TimelineStep(date='2024-01-15', value=9.5)  # Spike
# steps[24] = TimelineStep(date='2024-01-25', value=8.0)  # Second spike

# spikes = detect_spikes(steps, z_threshold=1.5)
# print(f'Detected {len(spikes)} spikes:')
# for s in spikes:
#     print(f'  [{s.rank}] {s.date}  Z={s.z_score:.2f}  vol={s.volume}')

Timeline built from 100 articles across 1 day(s):
  2026-02-27  articles=100

(Need ≥3 data points for spike detection — fetch more articles or widen the date range.)


## Test Actor Graph

In [14]:
from geoeventfusion.analysis.actor_graph import build_actor_graph

triples = [
    ('Houthi', 'United States', '2024-01-15'),
    ('Houthi', 'Yemen', '2024-01-15'),
    ('United States', 'United Kingdom', '2024-01-16'),
    ('Iran', 'Houthi', '2024-01-17'),
    ('Houthi', 'United States', '2024-01-18'),
    ('Iran', 'United States', '2024-01-19'),
]

graph = build_actor_graph(triples, hub_top_n=3, broker_ratio_threshold=0.5)
print(f'Nodes: {len(graph.nodes)}  Edges: {len(graph.edges)}')
print()
for node in sorted(graph.nodes, key=lambda n: n.pagerank, reverse=True)[:5]:
    print(f'  {node.name:<25} role={node.role:<12} pagerank={node.pagerank:.4f}')

Nodes: 5  Edges: 5

  Houthi                    role=Hub          pagerank=0.3193
  United States             role=Hub          pagerank=0.3193
  Iran                      role=Hub          pagerank=0.1657
  Yemen                     role=Peripheral   pagerank=0.0979
  United Kingdom            role=Peripheral   pagerank=0.0979


## Test Query Builder

In [ ]:
from geoeventfusion.analysis.query_builder import QueryBuilder

qb = QueryBuilder(
    base_query='Houthi Red Sea attacks',
    repeat_threshold=3,
    near_window=15,
    near_min_term_length=5,
    tone_negative_threshold=-5.0,
    toneabs_threshold=8.0,
)

# Build query variants
print('Repeat query:    ', qb.build_repeat_query())
print('Tone-neg query:  ', qb.build_tone_negative_query())
print('High-emotion:    ', qb.build_high_emotion_query())

## Test LLM Client

In [ ]:
from geoeventfusion.clients.llm_client import LLMClient

# Instantiate (no call made yet)
llm = LLMClient(
    backend=config.llm_backend,
    anthropic_model=config.anthropic_model,
    ollama_model=config.ollama_model,
    ollama_host=config.ollama_host,
    anthropic_api_key=config.anthropic_api_key,
    max_confidence=config.max_confidence,
)
print(f'LLMClient backend: {llm.backend}')
print(f'Max confidence cap: {llm.max_confidence}')

# Uncomment to make a live test call:
# response = llm.call(
#     system='You are a geopolitical analyst.',
#     prompt='In one sentence, what is the Houthi movement?',
#     max_tokens=100,
# )
# print('LLM response:', response)

## Inspect Fixture Data

In [ ]:
# Load and inspect the test fixtures
fixtures_dir = _ROOT / 'tests' / 'fixtures'

with open(fixtures_dir / 'sample_artlist.json', encoding='utf-8') as f:
    artlist = json.load(f)

articles = artlist.get('articles', [])
print(f'Fixture articles: {len(articles)}')
for a in articles[:3]:
    print(f'  [{a.get("seendate", "")}] {a.get("title", "")[:70]}')

## Run Full Pipeline (optional)

In [ ]:
# Uncomment to run the full pipeline with test fixtures (no real API calls)
# from geoeventfusion.pipeline import run_pipeline
# context = run_pipeline(config)
# print(f'Run ID: {context.run_id}')
# print(f'Warnings: {context.warnings}')
print('Uncomment the block above to run the full pipeline in test mode.')